In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
# 資料夾路徑（請確保結構為 train/men, train/women, test/men, test/women）
train_dir = 'train'
test_dir = 'test'

# 圖片大小與參數
img_height, img_width = 128, 128
batch_size = 32

# 🧼 僅標準化，不做任何增強
train_datagen = ImageDataGenerator(
    rescale=1./255,             # 基本正規化
    rotation_range=15,          # 隨機旋轉 ±15 度
    width_shift_range=0.1,      # 隨機水平平移 ±10%
    # height_shift_range=0.1,     # 隨機垂直平移 ±10%
    zoom_range=0.1,             # 隨機縮放 ±10%
    # horizontal_flip=True,       # 隨機水平翻轉（對臉部有效）
    fill_mode='nearest'         # 空白像素填補方式
)

test_datagen = ImageDataGenerator(rescale=1./255)

# 建立資料生成器
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

#🧠 CNN 模型（加上 L2 正則化）
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu',                  
                  input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(32, (3, 3), activation='relu'),                
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(1, activation='sigmoid')
])
early_stopping = EarlyStopping(
    monitor='accuracy',         # 監控驗證損失
    patience=1,                 # 容忍 3 個 epoch 沒進步
    restore_best_weights=True  # 回復到最佳的權重
)
# 編譯模型
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 🏋️‍♂️ 開始訓練
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)


Found 220 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Epoch 1/10
7/7 [==============================] - 16s 2s/step - loss: 1.1899 - accuracy: 0.5136 - val_loss: 0.7784 - val_accuracy: 0.5125
Epoch 2/10
7/7 [==============================] - 13s 2s/step - loss: 0.7709 - accuracy: 0.5227 - val_loss: 0.7656 - val_accuracy: 0.5125
Epoch 3/10
7/7 [==============================] - 13s 2s/step - loss: 0.7631 - accuracy: 0.5500 - val_loss: 0.7594 - val_accuracy: 0.5625
Epoch 4/10
7/7 [==============================] - 13s 2s/step - loss: 0.7581 - accuracy: 0.5318 - val_loss: 0.7507 - val_accuracy: 0.6125
Epoch 5/10
7/7 [==============================] - 13s 2s/step - loss: 0.7498 - accuracy: 0.5409 - val_loss: 0.7466 - val_accuracy: 0.5250
Epoch 6/10
7/7 [==============================] - 13s 2s/step - loss: 0.7503 - accuracy: 0.5636 - val_loss: 0.7350 - val_accuracy: 0.6250
Epoch 7/10
7/7 [==============================] - 13s 2s/step - loss: 0.7295 - accuracy: 0.